# SmartBeds - Preprocesado de Datos

## Grado en Ingeniería Informática
## Universidad de Burgos
##### José Luis Garrido Labrador

In [8]:
import pandas as pd # se importa pandas como pd
import numpy as np  #numpy como np

### Lectura

In [2]:
import loadData as ld

datos = ld.load(r"../data/")

datos = ld.preprocess(datos)

/home/jlgarridol/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


### Datos manuales

In [3]:
datos = ld.newSeizure(datos,'2018-11-10 3:30:00',60*20)

### Cálculo de las  estadísticas móviles
#### Widgets

In [4]:
import ipywidgets as widgets
from IPython.display import display

slider = widgets.IntSlider(
    value=25,
    min=1,
    max=50,
    description='Ventana:',
    readout=True,
    readout_format='d'
)

button = widgets.Button(
    description='Calcular'
)

progress = widgets.IntProgress(
    value=0,
    min=0,
    max=24,
    step=1,
    description='Procesado:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

hb = widgets.HBox([slider,button,progress])

In [5]:
display(hb)

dataE = datos.copy()

def estadisticos(b):
    global slider, dataE, progress
    
    progress.value = 0
    
    before = b.description
    b.description='Espera'

    print(" "*100,end='\r')
    
    ventanaMedia = slider.value
    
    for i in range(1,13):
        nombre_col = 'P'+str(i)
        nombre_col_mean = nombre_col+' mean'
        dataE[nombre_col_mean] = dataE[nombre_col].rolling(ventanaMedia).mean()
        progress.value += 1

    ventanaDesviacion = ventanaMedia

    for i in range(1,13):
        nombre_col = 'P'+str(i)
        nombre_col_std = nombre_col+' std'
        dataE[nombre_col_std] = dataE[nombre_col].rolling(ventanaDesviacion).std()
        progress.value += 1
        
    print('Calculos realizados para ventana a',ventanaMedia,end="\r")
    b.description = before
        
    dataE = dataE.dropna()
    

button.on_click(estadisticos)


### Exportación

In [6]:
import pickle as pk

In [7]:
with open('datos.pdd','wb') as f:
    pk.dump(datos,f)
    
with open('datosE.pdd','wb') as f:
    pk.dump(dataE,f)